<a href="https://colab.research.google.com/github/thomasavare/DNLP-project/blob/master/SPECTER_mag_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook aims at incorporating the MAG classification presented in the [Scidocs](https://github.com/allenai/scidocs) github. This classification consists of 19 different classes.

| int | Class label | int | Class label |
| --- | ----------- | --- | ----------- |
| 0   | Art         | 10	| History     |
| 1	  | Biology     | 11	| Materials science |
| 2	  | Business    | 12	| Mathematics |
| 3	  | Chemistry   | 13	| Medicine    |
| 4   |	Computer science | 14	| Philosophy  |
| 5	  | Economics   | 15	| Physics     |
| 6	  | Engineering | 16	| Political science |
| 7	  | Environmental science | 17 | Psychology  |
| 8	  | Geography   | 18	| Sociology   |
| 9	  | Geology     |

Everything said, let's begin.

First we import the [SPECTER](https://github.com/allenai/specter)embedding model. We are going to use the model accessible through the HuggingFace's transformer librairy.

In [1]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=08f9a2c2aa501b331d915473289de34df51ee9c6998ddcb757fe3daa56ecffc5
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer, util

# load model and tokenizer
model = SentenceTransformer('allenai-specter')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

We are now going to import the data used to train the models in the scidocs benchmarking. This include the meta-data (paper id, title, abstract and citation graph) and also the class associated to these paper.

In [3]:
!pip install awscli
!aws s3 sync --no-sign-request s3://ai2-s2-research-public/specter/scidocs/ data/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


download: s3://ai2-s2-research-public/specter/scidocs/mesh/test.csv to data/mesh/test.csv
download: s3://ai2-s2-research-public/specter/scidocs/mesh/train.csv to data/mesh/train.csv
download: s3://ai2-s2-research-public/specter/scidocs/mag/train.csv to data/mag/train.csv
download: s3://ai2-s2-research-public/specter/scidocs/cite/test.qrel to data/cite/test.qrel
download: s3://ai2-s2-research-public/specter/scidocs/coview/test.qrel to data/coview/test.qrel
download: s3://ai2-s2-research-public/specter/scidocs/cite/val.qrel to data/cite/val.qrel
download: s3://ai2-s2-research-public/specter/scidocs/cocite/val.qrel to data/cocite/val.qrel
download: s3://ai2-s2-research-public/specter/scidocs/coread/test.qrel to data/coread/test.qrel
download: s3://ai2-s2-research-public/specter/scidocs/coread/val.qrel to data/coread/val.qrel
download: s3://ai2-s2-research-public/specter/scidocs/cocite/test.qrel to data/cocite/test.qrel
download: s3://ai2-s2-research-public/specter/scidocs/mag/val.csv to d

In [4]:
import pandas as pd

train_path, test_path = '/content/data/mag/train.csv', '/content/data/mag/test.csv'
train_df, test_df = pd.read_csv(train_path), pd.read_csv(test_path)
train_df

,pid,class_label
0,78f6417635b95decc79e83d42812de50906a562c,10
1,bc636605d012c2a118e8390996843ffe9dc047b0,11
2,a093926d80986aaa93fdc609ce8b125954b87f6b,11
3,84dd81ec2622c433d26f8908d4bc1afd3bff8f6c,18
4,026a5dc5aeddd00b2323b2bedfc91b058c145934,17
...,...,...
17496,f648e027d210e68578d46501f14e581721f449aa,1
17497,78a254f0e8a03762cca5bbc0c7b0f39fc58229ca,2
17498,2eb64dd80227373760ac96fa08a46656e6516cd7,16
17499,2ecca4407aed9ecddac4578cd880e14f82600d0b,1


Unfortunatly, the HuggingFace SPECTER model isn't coinciding with the embeddings provided, so we have to recompute the embeddings. I did it once and dowloaded the data such that I will never have to do it again because it took 3 hours. Here's the code that I used.

----
```{python}
from tqdm import tqdm

for index, row in tqdm(data_df.iterrows(), total=len(data_df.index)):
  data_df['embedding'][row['paper_id']] = model.encode(row['title'] + '[SEP]' + row['title'])
```

100%|██████████| 48473/48473 [3:05:07<00:00, 4.36it/s]

----

The data is currently on my personal drive but I will upload it on my github so it can be easily imported and save 3 hours if someone is using that notebook one day.

In [5]:
embeddings_df = pd.read_json('/content/drive/MyDrive/DNLP-project/embeddings_metadata_mag_mesh.json', orient='index')

In [6]:
embeddings_df

,paper_id,title,abstract,embedding
00014a8515491f0b3fe2a1ff6e0f5305e584dcd9,00014a8515491f0b3fe2a1ff6e0f5305e584dcd9,ON THE CLASSIFICATION OF THE SCIENCES,None,"[-0.7367060184, 0.8094774485, -0.0390381552, -..."
00021eeee2bf4e06fec98941206f97083c38b54d,00021eeee2bf4e06fec98941206f97083c38b54d,Opportunities and challenges for E-Commerce in...,Numerous studies of E-Commerce have emphasized...,"[-1.1841081381, 1.1308963299, 0.6682422161, 0...."
00027baa2a90e1a3d50c1da14882d518de6847f5,00027baa2a90e1a3d50c1da14882d518de6847f5,Interactions between model membranes and ligni...,In order to elucidate the modes of interaction...,"[0.2389703691, 0.7384790778, 1.0119105577, -0...."
00034a5a5bd11b51ec046d31de273946d91fb766,00034a5a5bd11b51ec046d31de273946d91fb766,Perpetual Peace: What Kant Should Have Said,None,"[-0.7743704319, 0.5894224048, -0.3162544966, 0..."
000c8d85037886c86de15290e5a8e9bae7b66103,000c8d85037886c86de15290e5a8e9bae7b66103,Reimagining Greek Tragedy on the American Stage,List of Illustrations Preface Introduction CHA...,"[0.0163563173, 0.50049299, -0.0730320513, 0.64..."
...,...,...,...,...
ffe534bb74efd04b0038510be2a4ed5072430c8b,ffe534bb74efd04b0038510be2a4ed5072430c8b,Comparison of olmesartan combined with a calci...,The cardiovascular effects of combined therapy...,"[-0.5972291231, 0.17011463640000002, 0.6428002..."
ffeaac2b94fc298676e9784ed3bd7a6a7c23b9d1,ffeaac2b94fc298676e9784ed3bd7a6a7c23b9d1,Role of angiotensin II in plasma PAI-1 changes...,To evaluate the relationship between plasma pl...,"[-0.3963649869, 0.5153160095, 0.4946651161, -0..."
ffecb03cbf4dccf9fcdc8b26f40f53d5bc44be66,ffecb03cbf4dccf9fcdc8b26f40f53d5bc44be66,Factors that influence cancer patients' anxiet...,BACKGROUND\nNo study has yet assessed the impa...,"[-0.3526456356, 0.3627114594, -0.8083767891, 0..."
fff238844076ad5643dc2ff53153581bd89441ea,fff238844076ad5643dc2ff53153581bd89441ea,Is adjuvant chemotherapy indicated in stage I ...,OBJECTIVE\nConservative surgery followed by pl...,"[0.5204859972, -0.07988817990000001, -0.430343..."


Now we import the associated MAG classification and merge it with the embedding dataframe.

In [7]:
train_path, test_path = '/content/data/mag/train.csv', '/content/data/mag/test.csv'
train_df, test_df = pd.read_csv(train_path), pd.read_csv(test_path)
train_df.columns

Index(['pid', 'class_label'], dtype='object')

In [8]:
train_merged_df, test_merged_df = pd.merge(embeddings_df, train_df, how='inner', left_on='paper_id', right_on='pid'), pd.merge(embeddings_df, test_df, how='inner', left_on='paper_id', right_on='pid')
train_merged_df, test_merged_df = train_merged_df[['paper_id', 'embedding', 'class_label']], test_merged_df[['paper_id', 'embedding', 'class_label']]

In [9]:
train_merged_df

,paper_id,embedding,class_label
0,00021eeee2bf4e06fec98941206f97083c38b54d,"[-1.1841081381, 1.1308963299, 0.6682422161, 0....",2
1,00027baa2a90e1a3d50c1da14882d518de6847f5,"[0.2389703691, 0.7384790778, 1.0119105577, -0....",3
2,001d134f355306faeb8993fe487a34e0ed0d278b,"[-1.0588871241, 0.0980567485, 0.05070229990000...",10
3,001fb24b565bf8a5faf5551f753a55f3eed6c81c,"[-0.7206891179, 0.6086265445, -0.0693209022, -...",10
4,002087b2c4408d83bc095c835ce70e3974d830c7,"[0.0442002341, 0.0344819799, 0.408043295100000...",9
...,...,...,...
17496,ffeb51c4b44028bc1d5d71851f08d571c3177368,"[-0.3375469446, 0.5189490914, -0.2140062898, -...",12
17497,ffef3035f600d2e8cbb91192128d9315b800f71a,"[-0.33342462780000004, 0.8680065274000001, 0.2...",17
17498,fff1c7f9478f1220c9f9ea341e13651cbfc2996b,"[-1.2471318245, 0.6116715074, 0.36738637090000...",15
17499,fff73361b36bda4423c75f7e9a99e3996947a3c1,"[-1.0980789661, 0.3576785326, -0.2542884648, -...",16


Perfect. Now that we have our train and test data, we can focus on training different models.

Let's put the data in an adequat format.

In [10]:
import numpy as np

x_train, y_train = [], []
x_test, y_test = [], []

i = 0
for _, embedding, class_label in train_merged_df.values:
  x_train.append(embedding)
  y_train.append(class_label)

x_train = np.array(x_train)
y_train = np.array(y_train)

for _, embedding, class_label in test_merged_df.values:
  x_test.append(embedding)
  y_test.append(class_label)

x_test = np.array(x_test)
y_test = np.array(y_test)

First let's try a linear SVC.

In [11]:
from sklearn import svm

classifier1 = svm.LinearSVC(penalty='l2', loss='squared_hinge', max_iter=50000, random_state=42)

In [12]:
classifier1.fit(x_train, y_train)

LinearSVC(max_iter=50000, random_state=42)

In [13]:
from sklearn.metrics import f1_score

y_pred = classifier1.predict(x_test)
classifier1.score(x_test, y_test), f1_score(y_test, y_pred, average='macro')

(0.7307384697414023, 0.7209592925178011)

Let's put it to the test with different articles.

In [14]:
def classify_article(classifier, title, abstract, cls=False, probas=False):
  classes = {0: "Art", 1: "Biology", 2: "Business", 3: "Chemistry", 4: "Computer science", 5: "Economics", 6: "Engineering", 7: "Environmental science",\
             8: "Geography", 9: "Geology", 10: "History", 11: "Materials science", 12: "Mathematics", 13: "Medicine", 14: "Philosophy", 15: "Physics",\
             16: "Political science", 17: "Psychology", 18: "Sociology"}

  embed = model.encode([title + '[SEP]' + abstract])

  if cls:
    return np.array(list(zip(classifier.decision_function(embed)[0], classes.values())), dtype=object)
  return classes[classifier.predict(embed)[0]]

Title: SPECTER: Document-level Representation Learning using Citation-informed Transformers

Abstract: Representation learning is a critical ingredient for natural language processing systems. Recent Transformer language models like BERT learn powerful textual representations, but these models are targeted towards token- and sentence-level training objectives and do not leverage information on inter-document relatedness, which limits their document-level representation power. For applications on scientific documents, such as classification and recommendation, the embeddings power strong performance on end tasks. We propose SPECTER, a new method to generate document-level embedding of scientific documents based on pretraining a Transformer language model on a powerful signal of document-level relatedness: the citation graph. Unlike existing pretrained language models, SPECTER can be easily applied to downstream applications without task-specific fine-tuning. Additionally, to encourage further research on document-level models, we introduce SCIDOCS, a new evaluation benchmark consisting of seven document-level tasks ranging from citation prediction, to document classification and recommendation. We show that SPECTER outperforms a variety of competitive baselines on the benchmark.

Class: Computer science (obviously)


In [15]:
res = classify_article(classifier1, "SPECTER: Document-level Representation Learning using Citation-informed Transformers", "Representation learning is a critical ingredient for natural language processing systems. Recent Transformer language models like BERT learn powerful textual representations, but these models are targeted towards token- and sentence-level training objectives and do not leverage information on inter-document relatedness, which limits their document-level representation power. For applications on scientific documents, such as classification and recommendation, the embeddings power strong performance on end tasks. We propose SPECTER, a new method to generate document-level embedding of scientific documents based on pretraining a Transformer language model on a powerful signal of document-level relatedness: the citation graph. Unlike existing pretrained language models, SPECTER can be easily applied to downstream applications without task-specific fine-tuning. Additionally, to encourage further research on document-level models, we introduce SCIDOCS, a new evaluation benchmark consisting of seven document-level tasks ranging from citation prediction, to document classification and recommendation. We show that SPECTER outperforms a variety of competitive baselines on the benchmark.", cls=True)
res[np.argsort(-res[:, 0])]

array([[3.1524402822283992, 'Computer science'],
       [-1.2829318364693796, 'Geography'],
       [-1.819955043733155, 'Economics'],
       [-2.0848190048780912, 'Political science'],
       [-2.1363892816167187, 'Art'],
       [-2.3264718739329804, 'Sociology'],
       [-2.4614742849747677, 'History'],
       [-2.5346410455232027, 'Mathematics'],
       [-3.8988475683235815, 'Psychology'],
       [-3.9478657771508563, 'Environmental science'],
       [-4.241104738052026, 'Chemistry'],
       [-4.795763796057248, 'Philosophy'],
       [-5.118803121507698, 'Engineering'],
       [-5.125572656851713, 'Business'],
       [-5.5242928735337475, 'Geology'],
       [-5.614241875787205, 'Materials science'],
       [-6.230113152949974, 'Physics'],
       [-6.817697234456918, 'Biology'],
       [-7.510617406345038, 'Medicine']], dtype=object)

Successfuly classifies as computer science.

title: Exploring The Concept Of Cubism Art Essay

abstract: Cubism was one of the most influential art movements of the 20th century. It took place between 1907 and about 1914. The innovators of the Cubist movement were Pablo Picasso (Spanish, 1881 1973) and Georges Braque (French, 1882 1963). Cubism was one of the most significant changes in ideas in the history of art. It allowed for the development of many of the abstract modern art movements in areas such as Futurism and Constructivism.

Class: Art

In [16]:
res = classify_article(classifier1, "Exploring The Concept Of Cubism Art Essay", "Cubism was one of the most influential art movements of the 20th century. It took place between 1907 and about 1914. The innovators of the Cubist movement were Pablo Picasso (Spanish, 1881 1973) and Georges Braque (French, 1882 1963). Cubism was one of the most significant changes in ideas in the history of art. It allowed for the development of many of the abstract modern art movements in areas such as Futurism and Constructivism.", cls=True)
res[np.argsort(-res[:, 0])]

array([[0.702107422822192, 'Art'],
       [0.051804985553058236, 'Mathematics'],
       [-0.11936208911554957, 'Economics'],
       [-0.20883779260505306, 'Sociology'],
       [-0.6032807999713006, 'History'],
       [-2.024346498720357, 'Political science'],
       [-2.4536339165074805, 'Psychology'],
       [-2.5462747747625674, 'Computer science'],
       [-2.554336620674066, 'Philosophy'],
       [-2.645378385112636, 'Geography'],
       [-3.050846786161852, 'Physics'],
       [-4.0787498882182, 'Biology'],
       [-4.777605167710809, 'Engineering'],
       [-5.466593780576327, 'Geology'],
       [-5.590417764436368, 'Materials science'],
       [-5.8065289261875455, 'Medicine'],
       [-6.856526250836874, 'Business'],
       [-8.038728582816255, 'Chemistry'],
       [-9.74509356955713, 'Environmental science']], dtype=object)

Great success.

Let's use gridsearch to find the best set of parameters. The training might take a lot of time because it trains. We're also using f1 as a scoring function since it's the scoring function used in the scidoc benchmark.

The gridsearch is a quite long process when it has a lot of parameters. In this case, we test two different loss and 7 different C, so our gridsearch consist of training and comparing 14 different models to extract the best one.

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

parameters = {'loss': ('hinge', 'squared_hinge'), 'C': np.logspace(-4, 2, 7)}
svc = svm.LinearSVC()
clf1 = GridSearchCV(svc, parameters, scoring=make_scorer(f1_score, average="macro"))

In [18]:
clf1.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

GridSearchCV(estimator=LinearSVC(),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'loss': ('hinge', 'squared_hinge')},
             scoring=make_scorer(f1_score, average=macro))

In [31]:
params = clf1.get_params()
params

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': True,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__loss': 'squared_hinge',
 'estimator__max_iter': 1000,
 'estimator__multi_class': 'ovr',
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator': LinearSVC(),
 'n_jobs': None,
 'param_grid': {'loss': ('hinge', 'squared_hinge'),
  'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(f1_score, average=macro),
 'verbose': 0}

```
{'cv': None,
 'error_score': nan,
 'estimator': LinearSVC(),
 'n_jobs': None,
 'param_grid': {'loss': ('hinge', 'squared_hinge'),
  'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(f1_score, average=macro),
 'verbose': 0}
 ```

In [20]:
y_pred = clf1.predict(x_test)
clf1.score(x_test, y_test), np.mean(y_test == y_pred)

(0.7471634790922824, 0.7592641962143428)

f1 = 0.747

accuracy = 0.759

Not as good as in the scidocs benchmarking (f1 = 0.8195). But as we can see there's an error in the convergence and it advises to increase the number of iteration. So let's reuse the parameters of the best current model and increase the number of iterations. Right now, the maximum number of iterations is 1000, so let's set it to 50 000. All the other parameters are actually the default parameters.

In [22]:
clf1_2 = svm.LinearSVC(max_iter=50000)

In [23]:
clf1_2.fit(x_train, y_train)

LinearSVC(max_iter=50000)

In [26]:
y_pred = clf1_2.predict(x_test)
clf1_2.score(x_test, y_test), f1_score(y_test, y_pred, average='macro')

(0.7307384697414023, 0.7209592925178011)

Doesn't improve, tried to do a grid search on the C parameter with max_iter=50 000 but failed everytime and took about 7 hours so I'm not doing it.

Let's save the trained model into the Drive, maybe use it later for a more friendly user interface.

In [39]:
import pickle

filename = '/content/drive/MyDrive/DNLP-project/linearSVC_model1.sav'
pickle.dump(clf1, open(filename, 'wb'))

----

Let's try a non linear svm now.

In [40]:
from sklearn.model_selection import GridSearchCV

parameters = {'C': np.logspace(-4, 2, 7)}
svc = svm.SVC()
clf2 = GridSearchCV(svc, parameters, scoring=make_scorer(f1_score, average="macro"))

In [41]:
clf2.fit(x_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             scoring=make_scorer(f1_score, average=macro))

In [42]:
clf2.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(),
 'n_jobs': None,
 'param_grid': {'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(f1_score, average=macro),
 'verbose': 0}

```
{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(),
 'n_jobs': None,
 'param_grid': {'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(f1_score, average=macro),
 'verbose': 0}
```


In [43]:
y_pred = clf2.predict(x_test)
clf2.score(x_test, y_test), np.mean(y_test == y_pred)

(0.7432253137955992, 0.7525993068515062)

f1 = 0.743

accuracy = 0.752

no improvment, equivalent results.

In [44]:
filename = '/content/drive/MyDrive/DNLP-project/SVC_model1.sav'
pickle.dump(clf1, open(filename, 'wb'))

----

Let's try a CNN now. first we'll try a very simple CNN with only 2 dense layers.

In [58]:
import tensorflow as tf
y_train_cnn = tf.keras.utils.to_categorical(y_train)
x_train_cnn = np.expand_dims(x_train, -1)

In [60]:
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(x_train.shape[1], 1,), dtype="float")


# We add a vanilla hidden layer:
x = layers.Flatten()(inputs)
x = layers.Dense(x_train.shape[0], activation="relu")(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(19, activation="sigmoid", name="predictions")(x)

Cnn1 = tf.keras.Model(inputs, predictions)

In [61]:
Cnn1.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 768, 1)]          0         
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 dense_3 (Dense)             (None, 17501)             13458269  
                                                                 
 predictions (Dense)         (None, 19)                332538    
                                                                 
Total params: 13,790,807
Trainable params: 13,790,807
Non-trainable params: 0
_________________________________________________________________


In [62]:
Cnn1.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [63]:
epochs = 10

# Fit the model using the train and test datasets.
Cnn1.fit(x_train_cnn, y_train_cnn, epochs=epochs)

Epoch 1/10
547/547 [==============================] - 49s 89ms/step - loss: 1.0095 - accuracy: 0.6849
Epoch 2/10
547/547 [==============================] - 51s 94ms/step - loss: 0.8048 - accuracy: 0.7325
Epoch 3/10
547/547 [==============================] - 57s 104ms/step - loss: 0.7328 - accuracy: 0.7525
Epoch 4/10
547/547 [==============================] - 54s 98ms/step - loss: 0.6750 - accuracy: 0.7644
Epoch 5/10
547/547 [==============================] - 49s 90ms/step - loss: 0.6059 - accuracy: 0.7893
Epoch 6/10
547/547 [==============================] - 47s 85ms/step - loss: 0.5513 - accuracy: 0.8061
Epoch 7/10
547/547 [==============================] - 47s 86ms/step - loss: 0.4923 - accuracy: 0.8237
Epoch 8/10
547/547 [==============================] - 50s 91ms/step - loss: 0.4304 - accuracy: 0.8506
Epoch 9/10
547/547 [==============================] - 46s 84ms/step - loss: 0.3753 - accuracy: 0.8687
Epoch 10/10
547/547 [==============================] - 46s 84ms/step - loss: 0.33

In [76]:
x_test_cnn = np.expand_dims(x_test, -1)
y_pred = np.argmax(classifier.predict(x_test_cnn), axis=1)
np.mean(y_test == y_pred), f1_score(y_test, y_pred, average="macro")

118/118 [==============================] - 4s 33ms/step


(0.7272727272727273, 0.7195954609599928)

No big improvment compared to the SVC classification. Let's expand our network. This model is inspired by the "Alexnet" network and was initially designed for 1000 class classification and won the ImageNet Large Scale Visual Recognition Challenge in 2012.

In [105]:
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(x_train.shape[1], 1,), dtype="float")

x = layers.Conv1D(96, 11, 4)(inputs)
x = layers.MaxPool1D(3, 2)(x)
x = layers.Conv1D(256, 5)(x)
x = layers.MaxPool1D(3, 2)(x)
x = layers.Conv1D(384, 3)(x)
x = layers.Conv1D(384, 3)(x)
x = layers.MaxPool1D(3, 2)(x)

x = layers.Flatten()(x)
x = layers.Dense(2048, activation="relu")(x)
x = layers.Dense(2048, activation="relu")(x)


# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(19, activation="sigmoid", name="predictions")(x)

Cnn2 = tf.keras.Model(inputs, predictions)

In [106]:
Cnn2.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 768, 1)]          0         
                                                                 
 conv1d_52 (Conv1D)          (None, 190, 96)           1152      
                                                                 
 max_pooling1d_34 (MaxPoolin  (None, 94, 96)           0         
 g1D)                                                            
                                                                 
 conv1d_53 (Conv1D)          (None, 90, 256)           123136    
                                                                 
 max_pooling1d_35 (MaxPoolin  (None, 44, 256)          0         
 g1D)                                                            
                                                                 
 conv1d_54 (Conv1D)          (None, 42, 384)           295

In [107]:
Cnn2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [108]:
epochs = 10

# Fit the model using the train and test datasets.
Cnn2.fit(x_train_cnn, y_train_cnn, epochs=epochs)

Epoch 1/10
547/547 [==============================] - 224s 408ms/step - loss: 1.1695 - accuracy: 0.6319
Epoch 2/10
547/547 [==============================] - 236s 431ms/step - loss: 0.9128 - accuracy: 0.7074
Epoch 3/10
547/547 [==============================] - 227s 416ms/step - loss: 0.8588 - accuracy: 0.7181
Epoch 4/10
547/547 [==============================] - 227s 416ms/step - loss: 0.8197 - accuracy: 0.7297
Epoch 5/10
547/547 [==============================] - 222s 406ms/step - loss: 0.7720 - accuracy: 0.7429
Epoch 6/10
547/547 [==============================] - 225s 411ms/step - loss: 0.7476 - accuracy: 0.7488
Epoch 7/10
547/547 [==============================] - 221s 404ms/step - loss: 0.6999 - accuracy: 0.7625
Epoch 8/10
547/547 [==============================] - 224s 409ms/step - loss: 0.6498 - accuracy: 0.7787
Epoch 9/10
547/547 [==============================] - 225s 410ms/step - loss: 0.6570 - accuracy: 0.7826
Epoch 10/10
547/547 [==============================] - 223s 408m

In [109]:
x_test_cnn = np.expand_dims(x_test, -1)
y_pred = np.argmax(Cnn2.predict(x_test_cnn), axis=1)
np.mean(y_test == y_pred), f1_score(y_test, y_pred, average="macro")

118/118 [==============================] - 10s 85ms/step


(0.7118101839509464, 0.7015212521497566)

what a disappointment.